In [ ]:
import numpy as np
import pandas as pd
import math
import random
import csv

import pickle

from params import *
from functions import *

In [ ]:
f = open('results/results_online_SEP.txt', 'w+')

for SEED in range(0,10):

    set_seed(SEED)
    data_incoming = pd.read_pickle(f'data/data_incoming_{SEED}.pickle')
    lambda_results = pickle.load(open(f'results/results_offline_SEP_{SEED}.pickle','rb'))

    for passenger_percentage in [0, 0.2, 0.4, 0.6, 0.8, 1]:
        
        lambda_1 = lambda_results[passenger_percentage][0]
        lambda_2 = lambda_results[passenger_percentage][1]
        
        delta_objective_function_true_total = 0
        
        # track the remaining budget
        remaining_budget_passenger = B * passenger_percentage
        remaining_budget_driver = B * (1-passenger_percentage)

        n = 0
        
        while True:
            
            if (n >= len(data_incoming)):
                break

            fare = data_incoming.loc[n, 'fare']
            compensation = data_incoming.loc[n, 'compensation']
            
            if (fare*passenger_incentive_list[1] > remaining_budget_passenger and compensation*driver_incentive_list[1] > remaining_budget_driver):
                break

            l_best = -np.inf
            i_best = 0
            j_best = 0

            for i in range(0, len(passenger_incentive_list)):
                
                if (i > 0 and lambda_1 == np.inf):
                    continue
                
                for j in range(0, len(driver_incentive_list)):
                    
                    if (j > 0 and lambda_2 == np.inf):
                        continue

                    pred_prob = data_incoming.loc[n, f'pred_prob_{i}_{j}']

                    if (fare*passenger_incentive_list[i] <= remaining_budget_passenger and compensation*driver_incentive_list[j] <= remaining_budget_driver and fare*(1-passenger_incentive_list[i]) > compensation*(1+driver_incentive_list[j])):
                        
                        if (lambda_1 == np.inf and i == 0):
                            l_current = pred_prob - lambda_2*pred_prob*compensation*driver_incentive_list[j]
                        elif (lambda_2 == np.inf and j == 0):
                            l_current = pred_prob - lambda_1*pred_prob*fare*passenger_incentive_list[i]
                        else:
                            l_current = pred_prob - lambda_1*pred_prob*fare*passenger_incentive_list[i] - lambda_2*pred_prob*compensation*driver_incentive_list[j]
                            
                        if (l_current > l_best):
                            l_best = l_current
                            i_best = i
                            j_best = j

            delta_objective_function_true_total += data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] - data_incoming.loc[n, f'true_prob_0_0']
            remaining_budget_passenger -= data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] * fare*passenger_incentive_list[i_best]
            remaining_budget_driver -= data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] * compensation*driver_incentive_list[j_best]
            
            n += 1

        f.write(f'{SEED}, SEP, {passenger_percentage}, {delta_objective_function_true_total}, {n}\n')

f.close()